В этом ноутбуке мы протестируем каждую из двух моделей MobileBERT (одна обучена на spam sms, другая на spam emails) на каждом из датасетов (spam sms dataset и spam emails dataset). 

In [1]:
model_name = 'mobilebert'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install tensorflow-text
import tensorflow_text as text

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import load_model
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_text-2.11.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.8 MB)


In [4]:
spam_sms = 'spam sms'
spam_emails = 'spam emails'
mobilebert_trained_on_spam_sms = 'mobilebert trained on spam sms'
mobilebert_trained_on_spam_emails = 'mobilebert trained on spam emails'

In [5]:
models_dict = dict()
models_dict[mobilebert_trained_on_spam_sms] = load_model('/content/drive/MyDrive/data_for_colab/mobilebert_trained_on_spam_sms_20_january')
models_dict[mobilebert_trained_on_spam_emails] = load_model('/content/drive/MyDrive/data_for_colab/mobilebert_trained_on_spam_emails_20_january')

In [6]:
path_to_spam_sms_dataset = '/content/drive/MyDrive/data_for_colab/spam_sms.csv'
path_to_spam_emails_dataset = '/content/drive/MyDrive/data_for_colab/spam_emails.csv'

In [7]:
def test_model_on_dataset_and_write_results_to_dataframe(model_name=None, dataset_name=None, dataframe_to_write_answer=None):

    global spam_sms, spam_emails
    global mobilebert_trained_on_spam_sms
    global mobilebert_trained_on_spam_emails
    global models_dict
    global path_to_hotel_reviews_dataset
    global path_to_movie_reviews_dataset

    if model_name is None or dataset_name is None or dataframe_to_write_answer is None:
        raise ValueError("Wrong arguments passed to function: there are none arguments!")
    if model_name != mobilebert_trained_on_spam_sms and model_name != mobilebert_trained_on_spam_emails:
        raise ValueError("Wrong model_name!")
    if dataset_name != spam_sms and dataset_name != spam_emails:
        raise ValueError("Wrong dataset_name!")

    model = models_dict[model_name] # загрузили пользователем заданную модель 
    # (она уже обучена на определенном датасете)

    if dataset_name == spam_sms: # if you edit this line, edit the next line too!
        df = pd.read_csv(path_to_spam_sms_dataset, encoding = "ISO-8859-1")
        df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
        df.columns = ['IS_SPAM', 'DATA_COLUMN']
        df['IS_SPAM'] = (df['IS_SPAM'] == 'spam').astype(int)
        df_positive = df[df['IS_SPAM']==1]
        df_negative = df[df['IS_SPAM']==0]
        # Тестовая выборка
        n_test = df_negative.shape[0] // 2
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 2
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])

    elif dataset_name == spam_emails: # if you edit this line, edit the next line too!
        df = pd.read_csv(path_to_spam_emails_dataset, encoding = "ISO-8859-1")
        df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
        df.columns = ['DATA_COLUMN', 'IS_SPAM']
        df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' '))
        df_positive = df[df['IS_SPAM']==1]
        df_negative = df[df['IS_SPAM']==0]
        # Тестовая выборка
        n_test = df_negative.shape[0] // 2
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 2
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])

    X_test = df_balanced_test['DATA_COLUMN'].squeeze()
    y_test = df_balanced_test['IS_SPAM'].squeeze()

    y_predicted = model.predict(X_test)
    y_predicted = np.where(y_predicted > 0.5, 1, 0)

    row_name_in_dataframe_to_write_answer = model_name + ' tested on ' + dataset_name + ' dataset'

    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'accuracy'] = accuracy_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'precision'] = precision_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'recall'] = recall_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'f1_score'] =  f1_score(y_test, y_predicted)
    print(model_name, ' on dataset', dataset_name, 'result', accuracy_score(y_test, y_predicted), precision_score(y_test, y_predicted), recall_score(y_test, y_predicted), f1_score(y_test, y_predicted) )

In [8]:
answer_dataframe = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [9]:
for cur_model_name in [mobilebert_trained_on_spam_sms, mobilebert_trained_on_spam_emails]:
    for cur_dataset_name in [spam_sms, spam_emails]:
        test_model_on_dataset_and_write_results_to_dataframe(cur_model_name, cur_dataset_name, answer_dataframe)

88/88 [==============================] - 24s 214ms/step
mobilebert trained on spam sms  on dataset spam sms result 0.9813285457809695 0.9104859335038363 0.9544235924932976 0.9319371727748692
81/81 [==============================] - 12s 149ms/step
mobilebert trained on spam sms  on dataset spam emails result 0.48549323017408125 0.3109954456733897 0.6381842456608812 0.4181977252843394
88/88 [==============================] - 15s 143ms/step
mobilebert trained on spam emails  on dataset spam sms result 0.27899461400359066 0.11456859971711457 0.6514745308310992 0.19486768243785083
81/81 [==============================] - 12s 152ms/step
mobilebert trained on spam emails  on dataset spam emails result 0.9470019342359768 0.8713592233009708 0.9586114819759679 0.9129052765416401


In [10]:
answer_dataframe


,accuracy,precision,recall,f1_score
mobilebert trained on spam sms tested on spam sms dataset,0.981329,0.910486,0.954424,0.931937
mobilebert trained on spam sms tested on spam emails dataset,0.485493,0.310995,0.638184,0.418198
mobilebert trained on spam emails tested on spam sms dataset,0.278995,0.114569,0.651475,0.194868
mobilebert trained on spam emails tested on spam emails dataset,0.947002,0.871359,0.958611,0.912905


Сохраняем dataframe

In [11]:
name_for_dataframe = model_name + ' on spam testing result'

In [12]:
answer_dataframe.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_dataframe + '.csv')